In [1]:
#r "nuget:NEST"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages NEST, 7.17.4

In [2]:
#r "nuget:CsvHelper"

Installed Packages CsvHelper, 28.0.1

In [3]:
#r "nuget:Npgsql"

Installed Packages Npgsql, 6.0.6

In [4]:
using Nest;

In [5]:
using Elasticsearch.Net;

In [6]:
using CsvHelper;

In [7]:
using Newtonsoft.Json;

In [8]:
using Newtonsoft.Json.Converters;

In [9]:
using System;

In [10]:
using System.Globalization;

In [11]:
using System.Data;

In [12]:
using System.IO;

In [13]:
using System.Net;

In [14]:
using System.Security;

In [15]:
using System.Text;

In [16]:
using System.Collections.ObjectModel;

In [17]:
using System.Collections.Generic;

In [18]:
using System.Text.RegularExpressions;

In [19]:
using System.ComponentModel;

In [20]:
using System.Linq;

In [21]:
using Npgsql;

## DATE RANGE

In [22]:
DateTime StartDateTime = new DateTime(2022, 9, 1);
DateTime EndDateTime = new DateTime(2022, 9, 13);

## GLOBALS

In [23]:
Int32 numBeforeInsert = 0;
Int32 numAfterInsert = 0;
string sql = "";

## ACCEPT CERTIFICATE

In [24]:
System.Net.ServicePointManager.ServerCertificateValidationCallback += (sender_jf, cert_jf, chain_jf, errors_jf) => true;

## ENVIRONMENT

In [25]:
var F1 = System.Environment.GetEnvironmentVariable("f000");

var E1 = System.Environment.GetEnvironmentVariable("e100");
var W1 = System.Environment.GetEnvironmentVariable("e103");
var X1 = System.Environment.GetEnvironmentVariable("e104");
var Y1 = System.Environment.GetEnvironmentVariable("e107");

var D2 = System.Environment.GetEnvironmentVariable("g274");
var P2 = System.Environment.GetEnvironmentVariable("g270");
var W2 = System.Environment.GetEnvironmentVariable("g273");
var Y2 = System.Environment.GetEnvironmentVariable("g277");

## ELASTIC CONNECTION

In [26]:
var settings = new ConnectionSettings(new Uri(W1))
    .BasicAuthentication(E1,Y1)
    .ServerCertificateValidationCallback(CertificateValidations.AllowAll)
    .DefaultIndex(X1)
    .EnableDebugMode()
    .DisableDirectStreaming()
    .PrettyJson();

var elastic = new ElasticClient(settings);

## GRAPH DATABASE CONNECTION

In [27]:
string connString = @"Server=" + W2 + ";User ID=" + P2 + ";Password=" + Y2 + ";Database=" + D2 + ";";

NpgsqlConnection conn = new NpgsqlConnection(connString);

conn.Open();

// NpgsqlCommand cmd = new NpgsqlCommand("SET search_path = ag_catalog, \"$user\", public", conn);

// Int32 r = cmd.ExecuteNonQuery();

NpgsqlCommand cmd = new NpgsqlCommand("SET TIME ZONE UTC", conn);

Int32 r = cmd.ExecuteNonQuery();

## NODE: ACCOUNT

In [28]:
cmd = new NpgsqlCommand("DROP TABLE IF EXISTS node_account", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("CREATE TABLE IF NOT EXISTS node_account ("
                        + "id INT PRIMARY KEY"
                        + ", call UUID UNIQUE NOT NULL DEFAULT uuid_generate_v4()"
                        + ", userid TEXT UNIQUE NOT NULL"
                        + ")", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("DROP SEQUENCE IF EXISTS id_node_account_seq", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("CREATE SEQUENCE IF NOT EXISTS id_node_account_seq", conn);

r = cmd.ExecuteNonQuery();
    
cmd = new NpgsqlCommand("SELECT NEXTVAL('id_node_account_seq')", conn);

r = cmd.ExecuteNonQuery();

## NODE: DEVICE

In [29]:
cmd = new NpgsqlCommand("DROP TABLE IF EXISTS node_device", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("CREATE TABLE IF NOT EXISTS node_device ("
                        + "id INT PRIMARY KEY"
                        + ", call UUID UNIQUE NOT NULL DEFAULT uuid_generate_v4()"
                        + ", devicename TEXT UNIQUE NOT NULL"
                        + ", devicecity TEXT NOT NULL"
                        + ", devicedept TEXT NOT NULL"
                        + ", deviceserial TEXT NOT NULL"
                        + ", deviceid TEXT NOT NULL"
                        + ", deviceosmake TEXT NOT NULL"
                        + ", deviceosname TEXT NOT NULL"
                        + ", deviceosversion TEXT NOT NULL"
                        + ", deviceosnumber INT NOT NULL"
                        + ")", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("DROP SEQUENCE IF EXISTS id_node_device_seq", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("CREATE SEQUENCE IF NOT EXISTS id_node_device_seq", conn);

r = cmd.ExecuteNonQuery();
    
cmd = new NpgsqlCommand("SELECT NEXTVAL('id_node_device_seq')", conn);

r = cmd.ExecuteNonQuery();

## NODE: IPV4ADDRESS

In [30]:
cmd = new NpgsqlCommand("DROP TABLE IF EXISTS node_ipv4address", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("CREATE TABLE IF NOT EXISTS node_ipv4address ("
                        + "id INT PRIMARY KEY"
                        + ", call UUID UNIQUE NOT NULL DEFAULT uuid_generate_v4()"
                        + ", ipv4address TEXT UNIQUE NOT NULL"
                        + ", iso2 TEXT NOT NULL"
                        + ")", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("DROP SEQUENCE IF EXISTS id_node_ipv4address_seq", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("CREATE SEQUENCE IF NOT EXISTS id_node_ipv4address_seq", conn);

r = cmd.ExecuteNonQuery();
    
cmd = new NpgsqlCommand("SELECT NEXTVAL('id_node_ipv4address_seq')", conn);

r = cmd.ExecuteNonQuery();

## EDGE: PORTALAUTH

In [31]:
cmd = new NpgsqlCommand("DROP TABLE IF EXISTS edge_portalauth", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("CREATE TABLE IF NOT EXISTS edge_portalauth ("
                        + "id INT PRIMARY KEY"
                        + ", call UUID UNIQUE NOT NULL DEFAULT uuid_generate_v4()"
                        + ", logdatetimetz TIMESTAMPTZ NOT NULL"
                        + ", account TEXT NOT NULL"
                        + ", devicename TEXT NOT NULL"
                        + ", ipv4address TEXT NOT NULL"
                        + ", event TEXT NOT NULL DEFAULT 'unknown'"
                        + ", sequence BIGINT NOT NULL DEFAULT 0"
                        + ", method TEXT NOT NULL DEFAULT 'unknown'"
                        + ", status TEXT NOT NULL DEFAULT 'unknown'"
                        + ", error TEXT NOT NULL DEFAULT 'unknown'"
                        + ")", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("DROP SEQUENCE IF EXISTS id_edge_portalauth_seq", conn);

r = cmd.ExecuteNonQuery();

cmd = new NpgsqlCommand("CREATE SEQUENCE IF NOT EXISTS id_edge_portalauth_seq", conn);

r = cmd.ExecuteNonQuery();
    
cmd = new NpgsqlCommand("SELECT NEXTVAL('id_edge_portalauth_seq')", conn);

r = cmd.ExecuteNonQuery();

## CONVERT LIST TO DATATABLE

In [32]:
public static DataTable ListToDataTable<T>(IList<T> data)
{
    PropertyDescriptorCollection properties = TypeDescriptor.GetProperties(typeof(T));
    DataTable table = new DataTable();
    
    // Get properties
    foreach (PropertyDescriptor prop in properties)
        table.Columns.Add(prop.Name, Nullable.GetUnderlyingType(prop.PropertyType) ?? prop.PropertyType);
    
    // Row
    foreach (T item in data)
    {
        DataRow row = table.NewRow();
        // Properties = Columns
        foreach (PropertyDescriptor prop in properties)
            row[prop.Name] = prop.GetValue(item) ?? DBNull.Value;
        table.Rows.Add(row);
    }
    return table;
}

## INGRESS

In [33]:
public class IngressEvent_PortalAuth
{
    
    [Date(Name = "@timestamp")]
    public DateTime LogDateTime { get; set; }

    [Text(Name = "Event_ID")]
    public String Event { get; set; }
    
    [Text(Name = "Sequence_Number")]
    public UInt64 Sequence { get; set; }

    [Text(Name = "Authentication_Method")]
    public String Method { get; set; }
    
    [Text(Name = "Status")]
    public String Status { get; set; }

    [Text(Name = "Error_Code")]
    public String Error { get; set; }
    
    [Text(Name = "Source_User")]
    public String Account { get; set; }

    [Text(Name = "Public_IP")]
    public String IPV4Address { get; set; }

    [Text(Name = "Source_Region")]
    public String ISO2 { get; set; }

    [Text(Name = "Machine_Name")]
    public String DeviceName { get; set; }
    
    [Text(Name = "Serial_Number")]
    public String DeviceSerial { get; set; }

    [Text(Name = "Host_ID")]
    public String DeviceID { get; set; }
    
    [Text(Name = "Client_OS_Version")]
    public String DeviceOS { get; set; }

}

## EGRESS

In [34]:
public class AccountVertex
{

    [Text(Name = "chkEventLinkID")]
    public UInt64 chkEventLinkID { get; set; }
    
    [Text(Name = "Account")]
    public String Account { get; set; }

}

In [35]:
public class DeviceVertex
{
    
    [Text(Name = "chkEventLinkID")]
    public UInt64 chkEventLinkID { get; set; }
    
    [Text(Name = "DeviceName")]
    public String DeviceName { get; set; }
    
    [Text(Name = "DeviceCity")]
    public String DeviceCity { get; set; }
    
    [Text(Name = "DeviceDept")]
    public String DeviceDept { get; set; }
    
    [Text(Name = "DeviceSerial")]
    public String DeviceSerial { get; set; }

    [Text(Name = "DeviceID")]
    public String DeviceID { get; set; }
    
    [Text(Name = "DeviceOSMake")]
    public String DeviceOSMake { get; set; }
    
    [Text(Name = "DeviceOSName")]
    public String DeviceOSName { get; set; }
    
    [Text(Name = "DeviceOSVersion")]
    public String DeviceOSVersion { get; set; }
    
    [Text(Name = "DeviceOSNumber")]
    public String DeviceOSNumber { get; set; }

}

In [36]:
public class IPV4AddressVertex
{
    
    [Text(Name = "chkEventLinkID")]
    public UInt64 chkEventLinkID { get; set; }
    
    [Text(Name = "Public_IP")]
    public String IPV4Address { get; set; }

    [Text(Name = "Source_Region")]
    public String ISO2 { get; set; }

}

In [37]:
public class PortalAuthEdge
{
    
    [Date(Name = "LogDateTime")]
    public DateTime LogDateTime { get; set; }
    
    [Text(Name = "Account")]
    public String Account { get; set; }
    
    [Text(Name = "DeviceName")]
    public String DeviceName { get; set; }

    [Text(Name = "Public_IP")]
    public String IPV4Address { get; set; }
    
    [Text(Name = "chkEventLinkID")]
    public UInt64 chkEventLinkID { get; set; }

    [Text(Name = "Event")]
    public String Event { get; set; }
    
    [Text(Name = "Sequence")]
    public UInt64 Sequence { get; set; }

    [Text(Name = "Method")]
    public String Method { get; set; }
    
    [Text(Name = "Status")]
    public String Status { get; set; }

    [Text(Name = "Error")]
    public String Error { get; set; }

}

In [38]:
// Export 24 hours of data in 10k event chunks across many runs querying the same 24 hour span.
UInt64 RunID = 0;     // Tracks job query (runs), max 10k and zero based.
UInt64 EventID = 0;   // Max 10000 events per query run, zero based.

In [39]:
// UInt64 chkEventLinkID   // CSV Verification Field. Format: yyyyMMddrrrrreeeee Concatenate: LogDateTime RunID EventID

In [40]:
// UInt64 UtcNowDateInt64 = Convert.ToUInt64(DateTime.UtcNow.Date.ToString("yyyyMMdd"));

## EACH DAY

In [41]:
public IEnumerable<DateTime> EachDay(DateTime from, DateTime thru)
{
    for(var day = from.Date; day.Date <= thru.Date; day = day.AddDays(1))
        yield return day;
}

In [42]:
/*
foreach (DateTime day in EachDay(StartDate, EndDate))
    {
//        var Date = day.ToString("yyyy-MM-dd");
//        var StartDateTimeZone = day.ToString("yyyy-MM-dd") + "T00:00:00Z";
//        var EndDateTimeZone = day.ToString("yyyy-MM-dd") + "T23:59:59Z";
//        Console.WriteLine("Date: " + Date + " " + "TimeRange: " + StartDateTimeZone + ", " + EndDateTimeZone);
//        DateTime nextday = day.AddDays(1);
//        Console.WriteLine(nextday);
    }
*/

## TIME ZONE AMERICA/CHICAGO

In [43]:
// Set the ingress database time zone to the same time zone as the data source (Elastic)

cmd = new NpgsqlCommand("SET TIME ZONE 'America/Chicago'", conn);   
r = cmd.ExecuteNonQuery();

## MAIN

In [44]:
foreach (DateTime Start in EachDay(StartDateTime, EndDateTime))
{
    
// var StartDateTimeZone = day.ToString("yyyy-MM-dd") + "T00:00:00Z";
// var EndDateTimeZone = day.ToString("yyyy-MM-dd") + "T23:59:59Z";

// ONLY INCREMENT WHEN YOU HAVE MULTIPLE RUNS IN A 24 HOUR SPAN. -jf
// WHEN QUERY RESULTS FOR A SINGLE 24 HOUR SPAN ARE GREATER THAN 10,000 DOCUMENTS, THEN INCREMENT. -jf
// HOWEVER, IF QUERY RESULTS ARE LESS THAN 10,000 DOCUMENTS FOR A 24 HOUR SPAN THEN NO NEED TO INCREMENT. -jf
// RunID++; 
    
var End = Start.AddDays(1);
    
var FileDate = Start.ToString("yyyy-MM-dd");
    
var getLoginData = elastic
                .Search<IngressEvent_PortalAuth>(s => s
                    .Size(9000)
                    .Query(q => q
                        .Exists(e => e
                            .Field(f => f.Account)
                          ) && +q
                        .MatchPhrase(m => m
                            .Field(f => f.Event)
                            .Query("portal-auth")
                          ) && +q
                        .DateRange(r => r
                            .Field(f => f.LogDateTime)
                            .GreaterThanOrEquals(Start)
                            .LessThanOrEquals(End)
                            .Format("strict_date_optional_time")
                        )
                    )
                );



//  ACCOUNT VERTEX
    
    
    
EventID = 0;

List<AccountVertex> lstAccountData = new List<AccountVertex>();

foreach (var hit in getLoginData.Hits)
{

    if ( hit.Source.Account != null && hit.Source.DeviceOS != null && hit.Source.DeviceName != null )
    {
        
        lstAccountData.Add(new AccountVertex {
            chkEventLinkID = (((Convert.ToUInt64(hit.Source.LogDateTime.ToString("yyyyMMdd")) * 10000) + RunID) * 10000) + EventID,
            Account = Regex.Replace(hit.Source.Account, @"^(.*\\)(.*)$", "$2" )
        });
        
        EventID++;
    }
}


using (var writer_jf = new StreamWriter( F1 + FileDate + "Accounts.csv" ))
using (var csv_jf = new CsvWriter(writer_jf, CultureInfo.InvariantCulture ))
    csv_jf.WriteRecords(lstAccountData);


var dtAccountData = ListToDataTable(lstAccountData);
    
// Console.WriteLine(dtAccountData.Rows.Count);

    
numBeforeInsert = 0;
numAfterInsert = 0;
sql = "";

    
for (int i = 0; i < dtAccountData.Rows.Count; i++)
{
    cmd = new NpgsqlCommand("SELECT COUNT(*) FROM node_account", conn);
    numBeforeInsert = (Int32)(long) cmd.ExecuteScalar();
    
    sql = "INSERT INTO node_account (id, userid) VALUES ("
        + "(SELECT CURRVAL('id_node_account_seq')),"
        + "'" + dtAccountData.Rows[i]["Account"].ToString().Trim() + "')"
        + "ON CONFLICT (userid) DO NOTHING"
        + "; ";
    
    cmd = new NpgsqlCommand(sql, conn);
    r = cmd.ExecuteNonQuery();
    
    cmd = new NpgsqlCommand("SELECT COUNT(*) FROM node_account", conn);
    numAfterInsert = (Int32)(long) cmd.ExecuteScalar();
    
    if (numAfterInsert > numBeforeInsert) {
        cmd = new NpgsqlCommand("SELECT NEXTVAL('id_node_account_seq')", conn);
        r = cmd.ExecuteNonQuery();
    }
}

    
    
//  DEVICE VERTEX
    
    
    
EventID = 0;

List<DeviceVertex> lstDeviceData = new List<DeviceVertex>();

foreach (var hit in getLoginData.Hits)
{
    if ( hit.Source.Account != null && hit.Source.DeviceOS != null && hit.Source.DeviceName != null )
    {
        
        lstDeviceData.Add(new DeviceVertex {
            chkEventLinkID = (((Convert.ToUInt64(hit.Source.LogDateTime.ToString("yyyyMMdd")) * 10000) + RunID) * 10000) + EventID,
            DeviceName = hit.Source.DeviceName.ToUpper(),
            DeviceCity = Regex.Replace(hit.Source.DeviceName, @"^(.{3}).*$", "$1").ToUpper(),
            DeviceDept = Regex.Replace(hit.Source.DeviceName, @".*(.{4})$", "$1").ToUpper(),
            DeviceSerial = hit.Source.DeviceSerial,
            DeviceID = hit.Source.DeviceID,
            DeviceOSMake = Regex.Replace(hit.Source.DeviceOS, @"^.*?(\w+).*?$", "$1"),
            DeviceOSName = Regex.Replace(Regex.Replace(hit.Source.DeviceOS, @"^.*?(\w+).*?([^\d]+).*?$", "$2"), @"[\s]", ""),
            DeviceOSVersion = Regex.Replace(hit.Source.DeviceOS, @"^.*?([\d.]+).*?$", "$1"),
            DeviceOSNumber = Regex.Replace(Regex.Replace(hit.Source.DeviceOS, @"^.*?([\d.]+).*?$", "$1"), @"[.]", "")
        });
        
        EventID++;
    }
}


using (var writer_jf = new StreamWriter( F1 + FileDate + "Devices.csv" ))
using (var csv_jf = new CsvWriter(writer_jf, CultureInfo.InvariantCulture ))
    csv_jf.WriteRecords(lstDeviceData);


var dtDeviceData = ListToDataTable(lstDeviceData);
    
    
numBeforeInsert = 0;
numAfterInsert = 0;
sql = "";


for (int i = 0; i < dtDeviceData.Rows.Count; i++)
{
    cmd = new NpgsqlCommand("SELECT COUNT(*) FROM node_device", conn);
    numBeforeInsert = (Int32)(long) cmd.ExecuteScalar();
    
    sql = "INSERT INTO node_device ("
        + "id"
        + ", devicename"
        + ", devicecity"
        + ", devicedept"
        + ", deviceserial"
        + ", deviceid"
        + ", deviceosmake"
        + ", deviceosname"
        + ", deviceosversion"
        + ", deviceosnumber"
        + ") VALUES ("
        + "(SELECT CURRVAL('id_node_device_seq'))"
        + ", '" + dtDeviceData.Rows[i]["DeviceName"].ToString().Trim() + "'"
        + ", '" + dtDeviceData.Rows[i]["DeviceCity"].ToString().Trim() + "'"
        + ", '" + dtDeviceData.Rows[i]["DeviceDept"].ToString().Trim() + "'"
        + ", '" + dtDeviceData.Rows[i]["DeviceSerial"].ToString().Trim() + "'"
        + ", '" + dtDeviceData.Rows[i]["DeviceID"].ToString().Trim() + "'"
        + ", '" + dtDeviceData.Rows[i]["DeviceOSMake"].ToString().Trim() + "'"
        + ", '" + dtDeviceData.Rows[i]["DeviceOSName"].ToString().Trim() + "'"
        + ", '" + dtDeviceData.Rows[i]["DeviceOSVersion"].ToString().Trim() + "'"
        + ", " + dtDeviceData.Rows[i]["DeviceOSNumber"]
        + ")"
        + "ON CONFLICT (devicename) DO NOTHING"
        + "; ";
    
    cmd = new NpgsqlCommand(sql, conn);
    r = cmd.ExecuteNonQuery();
    
    cmd = new NpgsqlCommand("SELECT COUNT(*) FROM node_device", conn);
    numAfterInsert = (Int32)(long) cmd.ExecuteScalar();
    
    if (numAfterInsert > numBeforeInsert ) {
        cmd = new NpgsqlCommand("SELECT NEXTVAL('id_node_device_seq')", conn);
        r = cmd.ExecuteNonQuery();
    }
}    



// IPV4ADDRESS VERTEX
    
    
EventID = 0;

List<IPV4AddressVertex> lstIPV4AddressData = new List<IPV4AddressVertex>();

foreach (var hit in getLoginData.Hits)
{

    if ( hit.Source.Account != null && hit.Source.DeviceOS != null && hit.Source.DeviceName != null )
    {
        
        lstIPV4AddressData.Add(new IPV4AddressVertex {
            chkEventLinkID = (((Convert.ToUInt64(hit.Source.LogDateTime.ToString("yyyyMMdd")) * 10000) + RunID) * 10000) + EventID,
            IPV4Address = hit.Source.IPV4Address,
            ISO2 = hit.Source.ISO2
        });
        
        EventID++;
    }
}


using (var writer_jf = new StreamWriter( F1 + FileDate + "IPV4Addresses.csv" ))
using (var csv_jf = new CsvWriter(writer_jf, CultureInfo.InvariantCulture ))
    csv_jf.WriteRecords(lstIPV4AddressData);

    
var dtIPV4AddressData = ListToDataTable(lstIPV4AddressData);

    
numBeforeInsert = 0;
numAfterInsert = 0;
sql = "";


for (int i = 0; i < dtIPV4AddressData.Rows.Count; i++)
{
    cmd = new NpgsqlCommand("SELECT COUNT(*) FROM node_ipv4address", conn);
    numBeforeInsert = (Int32)(long) cmd.ExecuteScalar();
    
    sql = "INSERT INTO node_ipv4address (id, ipv4address, iso2) VALUES ("
        + "(SELECT CURRVAL('id_node_ipv4address_seq'))"
        + ", '" + dtIPV4AddressData.Rows[i]["IPV4Address"].ToString().Trim() + "'"
        + ", '" + dtIPV4AddressData.Rows[i]["ISO2"].ToString().Trim() + "'"
        + ")"
        + "ON CONFLICT (ipv4address) DO NOTHING"
        + "; ";
    
    cmd = new NpgsqlCommand(sql, conn);
    r = cmd.ExecuteNonQuery();
    
    cmd = new NpgsqlCommand("SELECT COUNT(*) FROM node_ipv4address", conn);
    numAfterInsert = (Int32)(long) cmd.ExecuteScalar();
    
    if (numAfterInsert > numBeforeInsert) {
        cmd = new NpgsqlCommand("SELECT NEXTVAL('id_node_ipv4address_seq')", conn);
        r = cmd.ExecuteNonQuery();
    }
}



// PORTALAUTH EDGE

    


EventID = 0;

List<PortalAuthEdge> lstPortalAuthData = new List<PortalAuthEdge>();

foreach (var hit in getLoginData.Hits)
{

    if ( hit.Source.Account != null && hit.Source.DeviceOS != null && hit.Source.DeviceName != null )
    {
        lstPortalAuthData.Add(new PortalAuthEdge {
            LogDateTime = hit.Source.LogDateTime,
            Account = Regex.Replace(hit.Source.Account, @"^(.*\\)(.*)$", "$2" ),
            DeviceName = hit.Source.DeviceName.ToUpper(),
            IPV4Address = hit.Source.IPV4Address,
            chkEventLinkID = (((Convert.ToUInt64(hit.Source.LogDateTime.ToString("yyyyMMdd")) * 10000) + RunID) * 10000) + EventID,
            Event = hit.Source.Event,
            Sequence = hit.Source.Sequence,
            Method = hit.Source.Method,
            Status = hit.Source.Status,
            Error = hit.Source.Error
        });
        
        EventID++;
    }
}


using (var writer_jf = new StreamWriter( F1 + FileDate + "PortalAuthEdges.csv" ))
using (var csv_jf = new CsvWriter(writer_jf, CultureInfo.InvariantCulture ))
    csv_jf.WriteRecords(lstPortalAuthData);

    
var dtPortalAuthData = ListToDataTable(lstPortalAuthData);

    
numBeforeInsert = 0;
numAfterInsert = 0;
sql = "";
    
    
for (int i = 0; i < dtPortalAuthData.Rows.Count; i++)
{
    cmd = new NpgsqlCommand("SELECT COUNT(*) FROM edge_portalauth", conn);
    numBeforeInsert = (Int32)(long) cmd.ExecuteScalar();
    
    sql = "INSERT INTO edge_portalauth ("
        + "id"
        + ", logdatetimetz"
        + ", account"
        + ", devicename"
        + ", ipv4address"
        + ", event"
        + ", sequence"
        + ", method"
        + ", status"
        + ", error"
        + ") VALUES ("
        + "(SELECT CURRVAL('id_edge_portalauth_seq'))"
        + ", '" + dtPortalAuthData.Rows[i]["LogDateTime"].ToString().Trim() + "'"
        + ", '" + dtPortalAuthData.Rows[i]["Account"].ToString().Trim() + "'"
        + ", '" + dtPortalAuthData.Rows[i]["DeviceName"].ToString().Trim() + "'"
        + ", '" + dtPortalAuthData.Rows[i]["IPV4Address"].ToString().Trim() + "'"
        + ", '" + dtPortalAuthData.Rows[i]["Event"].ToString().Trim() + "'"
        + ", '" + dtPortalAuthData.Rows[i]["Sequence"] + "'"
        + ", '" + dtPortalAuthData.Rows[i]["Method"].ToString().Trim() + "'"
        + ", '" + dtPortalAuthData.Rows[i]["Status"].ToString().Trim() + "'"
        + ", '" + dtPortalAuthData.Rows[i]["Error"].ToString().Trim() + "'"
        + ");";
    
    cmd = new NpgsqlCommand(sql, conn);
    r = cmd.ExecuteNonQuery();
    
    cmd = new NpgsqlCommand("SELECT COUNT(*) FROM edge_portalauth", conn);
    numAfterInsert = (Int32)(long) cmd.ExecuteScalar();
    
    if (numAfterInsert > numBeforeInsert) {
        cmd = new NpgsqlCommand("SELECT NEXTVAL('id_edge_portalauth_seq')", conn);
        r = cmd.ExecuteNonQuery();
    }
}
    
    
}   // Main foreach

// RunID++; // More jobs... if needed to extract 10k records at a time.

## UTC

In [45]:
cmd = new NpgsqlCommand("SET TIME ZONE UTC", conn);   
r = cmd.ExecuteNonQuery();

In [46]:
/*********************************************/